# Time Series Forecasting

### Imports

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import dates
import joblib
import statsmodels.api as sm

from tkinter import *
import dtale as dt
import plotly.graph_objects as go

from datetime import datetime, timedelta

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

pd.set_option('display.float_format', '{:,.3}'.format)
# pd.set_option('max_columns', 100)


/home/fqp/anaconda3/envs/s1_v3/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


### 1.] Preparations

#### Loading & Preparing the Data

In [2]:
df_sys = joblib.load("./data/GTMA Trades_df_sys")
df_enr = joblib.load("./data/GTMA Trades_df_enr")

# df_sys.head(6)

In [3]:

#? Resampling to Daily frequency.
df_sys_D1 = df_sys.set_index(['End Time_D&T'])[['Volume', 'Cost']].resample('D').sum()
df_sys_D2 = df_sys.set_index(['End Time_D&T'])[['Price', 'Trade Hours Duration']].resample('D').mean()
df_sys_D = df_sys_D1.join(df_sys_D2,how='left')
# df_sys_D.head(4)

#ToDo: PowerTransformer here after Resampling.

In [4]:
df_enr_D1 = df_enr.set_index(['End Time_D&T'])[['Volume', 'Cost']].resample('D').sum()
df_enr_D2 = df_enr.set_index(['End Time_D&T'])[['Price', 'Trade Hours Duration']].resample('D').mean()
df_enr_D = df_enr_D1.join(df_enr_D2,how='left')
# df_enr_D.head(4)

In [ ]:

dt.show(df_sys_D, open_browser=True, ignore_index=True)

In [ ]:
dt.show(df_enr_D, open_browser=True, ignore_index=True)

In [5]:

#? Reloading the .csv after removing rows
df_sys_D = pd.read_csv("./data/df_sysD_19-21.csv")
df_sys_D.head(4)

,End Time_D&T,Volume,Cost,Price,Trade Hours Duration
0,2019-01-01,-8.03e+03,3.45e+05,39.2,3.74
1,2019-01-02,-9.9e+02,-3.14e+04,33.9,3.33
2,2019-01-03,0.0,0.0,NaN,NaN
3,2019-01-04,0.0,0.0,NaN,NaN


In [ ]:
df_enr_D = pd.read_csv("./data/df_enrD_19-21.csv")
joblib.dump(df_enr_D,"./data/GTMA Trades_df_enrD_19-21")

In [ ]:
joblib.dump(df_sys_D,"./data/GTMA Trades_df_sysD_19-21")

In [2]:
df_sysD_19to21 = joblib.load('./data/GTMA Trades_df_sysD_19-21')
# df_sysD_19to21.head(4)

In [3]:
df_enrD_19to21 = joblib.load('./data/GTMA Trades_df_enrD_19-21')

In [4]:

df_sysD_19to21["End Time_D&T"] = pd.to_datetime(df_sysD_19to21["End Time_D&T"], format="%Y-%m-%d")
# df_sysD_19to21.head(6)

In [ ]:

#? Checking datatypes.
df_sysD_19to21.dtypes

In [5]:
df_enrD_19to21["End Time_D&T"] = pd.to_datetime(df_enrD_19to21["End Time_D&T"], format="%Y-%m-%d")

> Scaling the Formatted df

In [6]:
transformer_pwr = joblib.load('./transformer_pwr')

In [7]:

# standardized = transformer_pwr.fit_transform(df_sysD_19to21["Cost"]).reshape(-1)
df_sysD_19to21["Cost_pt"] = transformer_pwr.fit_transform(df_sysD_19to21[["Cost"]]).reshape(-1)

# standardized = transformer_pwr.fit_transform(df_enrD_19to21["Cost"]).reshape(-1)
df_enrD_19to21["Cost_pt"]= transformer_pwr.fit_transform(df_enrD_19to21[["Cost"]]).reshape(-1)

#### Auxillary Functions

In [8]:
dict_train = {
        "Y2019_Q1" : 0,               #? Start of 2019
        "Y2019_Q2" : 89,              
        "Y2019_Q3" : 182,             #? 2019 Midyear
        "Y2019_Q4" : 182 + 92,
        "Y2020_Q1" : 365,            #? Start of 2020
        "Y2020_Q2" : 365 + 89,       
        "Y2020_Q3" : 365 + 182,      #? 2020 midyear, Start of July     
        "Y2020_Q4" : 365 + 182 + 92  
        }

layout = go.Layout(
        margin=go.layout.Margin(
        l=20, #left margin
        r=15, #right margin
        b=15, #bottom margin
        t=35 ),#top margin
        xaxis_title="Date", yaxis_title="Cost (£)")

In [9]:

def go_plotter(df_train, df_test, df_forecast, FC, LABEL, CAT):
        fig = go.Figure(layout=layout)
        fig.update_layout(title= "2021 "+ str(CAT) + " Trades " + str(FC) + "-Day Forecast" + f"<br><sup>Train -- {LABEL}</sup>")

        #? Note: Index of incoming dataframes are Datetime
        for i, t in zip([df_train, df_test, df_forecast],["Train","Test","Forecast"]):
                mode_var = 'lines+markers' if (t =="Forecast") else 'lines'
                line_dash = 'dot' if (t =="Train") else 'solid'
                fig.add_trace(go.Scatter(x=i.index, y=i["Cost_pt"].values, name=t, mode=mode_var,
                line_dash=line_dash, hovertemplate="Date = %{x}<br>Cost_pt = %{y}"))
        # fig.for_each_trace(lambda trace: trace.update(fill='tonextx') if trace.name == "Forecast" else ())
        fig.show()

In [10]:
EPSILON = 1e-10
def maape(actual, predicted):
    #? Mean Arctangent Absolute Percentage Error
    #? Note: result is NOT multiplied by 100
    return np.mean(np.arctan(np.abs((actual - predicted) / (actual + EPSILON))))

In [ ]:

#ToDo: Not used. Needs to be updated.
def bokeh_plotter(train,test,forecast):
    #? Specific only for this project: Because of rigidly-named variables.
    
    #? ds_ == "data source", as required by bokeh plots
    ds_train = ColumnDataSource(train)
    ds_test = ColumnDataSource(test)
    ds_forecast = ColumnDataSource(forecast)

    TOOLTIPS = [('Date', '@{End Time_D&T}{%F}'),
                ('Cost', '@Cost{($ 0.00 a)}')]
    
    hover = HoverTool(tooltips=TOOLTIPS, 
                    formatters={
                        '@{End Time_D&T}' : 'datetime',
                            },
                    mode="mouse")
    
    #? Establishing plot
    p = figure(x_axis_type="datetime", plot_width=1100, plot_height=420)
    p.add_tools(hover)

    #? Plot configurations
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    p.xaxis.axis_label = 'Date'
    p.yaxis.axis_label = 'Cost'
    
    #* Main plots
    line_train = p.line(x='End Time_D&T', y='Cost', source=ds_train,color="grey", line_alpha=.35)
    line_test = p.line(x='End Time_D&T', y='Cost', source=ds_test,color="orange", line_dash="dotted",legend_label="Test",line_width=2)
    line_forecast = p.line(x='End Time_D&T', y='Cost', source=ds_forecast, legend_label="ExpoSmooth Forecast -- System Trade")
    
    #? Configs must be AFTER Main plots 
    p.toolbar.autohide = True
    p.legend.click_policy="hide"

    show(p)

### 2.] Exponential Smoothing

#### System Trades

In [63]:

#? CAT = "System"
LABEL = "Y2019_Q1"

FC = 60

#? (-152) signifies up until the end of 2020.
df_train = df_sysD_19to21[['End Time_D&T','Cost_pt']][dict_train.get(LABEL):-152].set_index("End Time_D&T", drop=True)
df_test = df_sysD_19to21[['End Time_D&T',"Cost_pt"]].set_index("End Time_D&T", drop=True)
df_test = df_test[["Cost_pt"]][df_train.index[-1]+timedelta(days=1): (df_train.index[-1] + timedelta(days=(FC)))]


In [64]:

#* Daily Frequency

#? Arguments for less than two years Training period: seasonal_periods=30, seasonal='add'
model_fit_exposmo_sysD = ExponentialSmoothing(df_train, trend='add', damped_trend=True, freq='D',
                                initialization_method="estimated",seasonal_periods= 365 if LABEL == "Y2019_Q1" else 30, seasonal='add').fit(optimized=True)

df_forecast = pd.DataFrame(model_fit_exposmo_sysD.forecast(len(df_test.index)).values, 
                        index=df_test.index, columns=['Cost_pt'])

# fig.update_layout(title="System Trades " + str(FC) + "-Day Forecast" + " -- " + LABEL)


In [65]:
go_plotter(df_train, df_test, df_forecast, FC, LABEL, "System")
maape(df_test.values, df_forecast.values)

0.7767950011682095

In [ ]:

#? Basis for Bokeh plotting
model_fit_exposmo_sysD = ExponentialSmoothing(df_sysD_19to21['Cost'][(365+182+1):-152], seasonal_periods=30, trend='add', seasonal='add',
                                    damped_trend=True, initialization_method="heuristic").fit(optimized=True)

df_forecast = pd.DataFrame(model_fit_exposmo_sysD.forecast(60).values, 
                            index=df_sysD_19to21["End Time_D&T"][-152:-92], columns=['Cost'])

bokeh_plotter(df_train,
            df_test, 
            df_forecast)

In [ ]:
joblib.dump(model_sarima_sysM,"./model_sarima_sysM_Y2018S_201x100")

#### Energy Trades

In [45]:

LABEL = "Y2020_Q3"

FC = 30

df_train = df_enrD_19to21[['End Time_D&T','Cost_pt']][dict_train.get(LABEL):-152].set_index("End Time_D&T", drop=True)
df_test = df_enrD_19to21[['End Time_D&T',"Cost_pt"]].set_index("End Time_D&T", drop=True)
df_test = df_test[["Cost_pt"]][df_train.index[-1]+timedelta(days=1): (df_train.index[-1] + timedelta(days=(FC)))]

In [46]:

#? Optional arguments: seasonal_periods=30, seasonal='add'
model_fit_exposmo_enrD = ExponentialSmoothing(df_train, trend='add', damped_trend=True, freq='D',
                                initialization_method="heuristic",seasonal_periods=365 if LABEL == "Y2019_Q1" else 30, seasonal='add').fit(optimized=True)

df_forecast = pd.DataFrame(model_fit_exposmo_enrD.forecast(len(df_test.index)).values, 
                        index=df_test.index, columns=['Cost_pt'])


In [47]:
go_plotter(df_train, df_test, df_forecast, FC, LABEL, "Energy")
maape(df_test.values, df_forecast.values)

0.8753486640958924

_______________________

### 3.] References

1.) MAAPE -- "Thus, MAAPE is scale-independent, can be interpreted intuitively as an absolute percentage error, and is simple to calculate." [https://www.sciencedirect.com/science/article/pii/S0169207016000121]

2.) Forecasting metrics -- forecasting_metrics.py, https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9

3.) Ignoring warnings -- https://stackoverflow.com/questions/34444607/how-to-ignore-statsmodels-maximum-likelihood-convergence-warning